# Pipeline ETL - API → MongoDB Atlas

In [62]:
!pip install -q pymongo python-dotenv requests pandas dnspython

In [63]:
import os
from typing import List

from fastapi import FastAPI, Header, HTTPException, Depends
from pydantic import BaseModel
from pymongo import MongoClient, UpdateOne

## 1️⃣ Configuração

In [64]:
import os
import requests
import pandas as pd
from pymongo import MongoClient

CONFIG_URL = "https://r-config-service.up.railway.app/config"

def carregar_config_seguro():
    resp = requests.get(CONFIG_URL, timeout=10)
    resp.raise_for_status()
    data = resp.json()

    api_key = data.get("API_KEY")
    atlas_uri = data.get("ATLAS_URI")

    if not api_key or not atlas_uri:
        raise RuntimeError("❌ r-config-service não retornou todas as variáveis")

    print("✅ Variáveis recebidas do r-config-service")
    return api_key, atlas_uri

def conectar_mongodb(atlas_uri):
    client = MongoClient(atlas_uri)
    db = client["test"]
    print("✅ Conexão com MongoDB Atlas estabelecida")
    return db

if __name__ == "__main__":
    api_key, atlas_uri = carregar_config_seguro()
    db = conectar_mongodb(atlas_uri)

    collection = db["users"]
    registros = list(collection.find({}, {"_id": 0}))
    print("Registros:", registros)



✅ Variáveis recebidas do r-config-service
✅ Conexão com MongoDB Atlas estabelecida
Registros: [{'id': 4, 'name': 'Pyterson', 'account': {'id': 7, 'number': '00001-1', 'agency': '0001', 'balance': 0, 'limit': 500}, 'card': {'id': 4, 'number': '**** **** **** 1111', 'limit': 1000}, 'features': [], 'news': [{'id': 9, 'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg', 'description': 'Pyterson, invista hoje para garantir um futuro seguro.'}], '__v': 0}, {'id': 5, 'name': 'Pip', 'account': {'id': 8, 'number': '00002-2', 'agency': '0001', 'balance': 0, 'limit': 500}, 'card': {'id': 5, 'number': '**** **** **** 2222', 'limit': 1000}, 'features': [], 'news': [{'id': 10, 'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg', 'description': 'Invista hoje para um futuro seguro e estável, Pip.'}], '__v': 0}, {'id': 6, 'name': 'Pep', 'account': {'id': 9, 'number': '00003-3', 'agency': '0001', 'balance': 0, 'limit': 5

## 2️⃣ Extract

In [ ]:
## https://users-api-etl.up.railway.app/api-docs/

In [65]:
df = pd.read_csv('SDW2023.csv')
user_ids = df["UserID"].tolist()
print(user_ids)

[3, 4, 5]


In [ ]:
import requests
import json

API_URL = "https://users-api-etl.up.railway.app/users"

def get_user(id):
    response = requests.get(f'{API_URL}/{id}') #data = response.json()
    return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

## 3️⃣ Transform

In [ ]:
!pip install openai

In [69]:
from openai import OpenAI
import os

client = OpenAI(
    base_url="https://api.tokenfactory.nebius.com/v1/",
    api_key=os.environ["NEBIUS_API_KEY"]
)

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-fast",
    messages=[
        {"role": "system", "content": "Você é um assistente útil."},
        {"role": "user", "content": "Responda apenas: OK"}
    ],
    temperature=0
)

print(response.choices[0].message.content)


OK


In [70]:

import os
from openai import OpenAI

client = OpenAI(
    base_url="https://api.tokenfactory.nebius.com/v1/",
    api_key=os.environ["NEBIUS_API_KEY"]
)

def generate_ai_news(user):
    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-fast",
        messages=[
            {
                "role": "system",
                "content": "Você é um especialista em marketing bancário."
            },
            {
                "role": "user",
                "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)."
            }
        ],
        temperature=0.7,
        max_tokens=60
    )

    return response.choices[0].message.content.strip('"')

for user in users:
    news = generate_ai_news(user)
    print(news)

    # garante que a chave 'news' exista
    if "news" not in user or user["news"] is None:
        user["news"] = []

    user["news"].append({
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": news
    })



Ana, investir é proteger seu dinheiro e construir seu futuro. Não perca a chance de crescer seu patrimônio!
Conheça Pyterson e comece a construir seu futuro financeiro com investimentos! Cresça seu patrimônio e garantir sua segurança financeira.
Pip, sua economia cresce enquanto você investe! Aprenda a juntar e a multiplicar seu dinheiro para um futuro mais seguro e próspero!


## 4️⃣ Load

In [71]:
# Atualizar os usuários na API do desafio com os dados enriquecidos

import requests

API_URL = "https://users-api-etl.up.railway.app/users"

def update_user(user):
    response = requests.put(
        f"{API_URL}/{user['id']}",
        json=user
    )

    if response.status_code != 200:
        print("STATUS:", response.status_code)
        print("BODY:", response.text)

    return response.status_code == 200


for user in users:
    success = update_user(user)
    print(f"User {user['name']} updated? {success}!")




User Ana Pereira updated? True!
User Pyterson updated? True!
User Pip updated? True!


## 5️⃣ Validação

In [72]:

from pprint import pprint
docs = list(collection.find({}, {"_id": 0}).limit(8))
pprint(docs)


[{'__v': 0,
  'account': {'agency': '0001',
              'balance': 0,
              'id': 7,
              'limit': 500,
              'number': '00001-1'},
  'card': {'id': 4, 'limit': 1000, 'number': '**** **** **** 1111'},
  'features': [],
  'id': 4,
  'name': 'Pyterson',
  'news': [{'description': 'Pyterson, invista hoje para garantir um futuro '
                           'seguro.',
            'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg',
            'id': 9},
           {'description': 'Conheça Pyterson e comece a construir seu futuro '
                           'financeiro com investimentos! Cresça seu '
                           'patrimônio e garantir sua segurança financeira.',
            'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg'}]},
 {'__v': 0,
  'account': {'agency': '0001',
              'balance': 0,
              'id': 8,
              'limit': 500,
              'n